# 🚀 CARRIER PRODUCTION — Full Pipeline Execution

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                 FRÉGATE 06_AIRCRAFT_CARRIER — PRODUCTION MODE                ║
║          Assembly • Audio Sync • RIFE 120FPS • 4K Upscale • Encode           ║
╚══════════════════════════════════════════════════════════════════════════════╝
```

**Mission**: Produire le livrable final 4K/120FPS à partir des composants.

---

## 📦 Setup Production Environment

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Configuration
DRIVE_ROOT = "/content/drive/MyDrive/DRIVE_EXODUS_V2"
UNIT_ROOT = f"{DRIVE_ROOT}/06_AIRCRAFT_CARRIER"
CODEBASE = f"{UNIT_ROOT}/CODEBASE"

import sys
sys.path.insert(0, CODEBASE)

print("╔" + "═" * 50 + "╗")
print("║   AIRCRAFT CARRIER PRODUCTION — EXODUS SYSTEM     ║")
print("╚" + "═" * 50 + "╝")
print(f"\n✓ Environment configured")
print(f"✓ Drive Root: {DRIVE_ROOT}")

In [ ]:
# Install dependencies
!apt install -y ffmpeg
!pip install -q torch torchvision opencv-python-headless scipy Pillow tqdm

# Optionnel: Real-ESRGAN pour upscale IA
# !pip install -q basicsr realesrgan

## 📋 Configuration Production

In [ ]:
# Configuration du projet
PRODUCTION_CONFIG = {
    "project_name": "EXODUS_EPISODE_01",
    
    # Chemins (laisser vide pour défauts)
    "components_dir": "",  # Défaut: IN_COMPONENTS/
    "output_dir": "",       # Défaut: OUT_FINAL/
    "production_plan": "", # Défaut: IN_COMPONENTS/PRODUCTION_PLAN.JSON
    
    # Options pipeline
    "use_rife": True,      # False = FFmpeg minterpolate
    "use_upscale": True,   # False = skip upscale
    "use_gpu": True,       # False = CPU only
    
    # Output
    "target_fps": 120,
    "target_resolution": "4K",  # "4K", "1080p"
    "codec": "h265",       # "h265", "prores"
}

print("=== Configuration Production ===")
for key, val in PRODUCTION_CONFIG.items():
    print(f"  {key}: {val}")

## 🔍 Validation Pre-Flight

In [ ]:
from pathlib import Path
import json

def validate_production():
    """Valide que tout est prêt pour la production."""
    errors = []
    warnings = []
    
    # Chemins
    components_dir = Path(PRODUCTION_CONFIG.get('components_dir') or f"{UNIT_ROOT}/IN_COMPONENTS")
    output_dir = Path(PRODUCTION_CONFIG.get('output_dir') or f"{UNIT_ROOT}/OUT_FINAL")
    
    if not components_dir.exists():
        errors.append(f"Components dir non trouvé: {components_dir}")
    
    # Séquences
    exr_files = list(components_dir.glob("*.exr"))
    png_files = list(components_dir.glob("*.png"))
    
    if not exr_files and not png_files:
        errors.append("Aucune séquence image (EXR/PNG) trouvée")
    else:
        seq_count = len(exr_files) or len(png_files)
        print(f"✓ Séquences trouvées: {seq_count} images")
    
    # Audio
    wav_files = list(components_dir.glob("*.wav"))
    if not wav_files:
        warnings.append("Aucun audio trouvé - vidéo sera muette")
    else:
        print(f"✓ Audio trouvé: {len(wav_files)} pistes")
    
    # PRODUCTION_PLAN
    plan_path = Path(PRODUCTION_CONFIG.get('production_plan') or f"{components_dir}/PRODUCTION_PLAN.JSON")
    if plan_path.exists():
        with open(plan_path) as f:
            plan = json.load(f)
        output_cfg = plan.get('output', {})
        print(f"✓ Plan trouvé: {output_cfg.get('resolution', '?')} @ {output_cfg.get('framerate', '?')}FPS")
    else:
        warnings.append(f"PRODUCTION_PLAN.JSON non trouvé, défauts utilisés")
    
    # RIFE
    if PRODUCTION_CONFIG.get('use_rife'):
        rife_path = Path(f"{DRIVE_ROOT}/EXODUS_AI_MODELS/rife")
        if not rife_path.exists():
            warnings.append("RIFE non trouvé, FFmpeg fallback sera utilisé")
        else:
            print(f"✓ RIFE modèle trouvé")
    
    # GPU
    if PRODUCTION_CONFIG.get('use_gpu'):
        try:
            import torch
            if torch.cuda.is_available():
                print(f"✓ GPU disponible: {torch.cuda.get_device_name(0)}")
            else:
                warnings.append("CUDA non disponible, fallback CPU")
        except ImportError:
            warnings.append("PyTorch non installé, fallback CPU")
    
    # Résumé
    print("\n" + "=" * 50)
    if errors:
        print("❌ VALIDATION ÉCHOUÉE")
        for e in errors:
            print(f"   ERROR: {e}")
    else:
        print("✅ VALIDATION OK")
    
    if warnings:
        print("\n⚠ Warnings:")
        for w in warnings:
            print(f"   {w}")
    
    return len(errors) == 0

validation_ok = validate_production()

## 🏭 Exécution Production

In [ ]:
import subprocess
import time
from datetime import datetime

def run_carrier():
    """Exécute le pipeline CARRIER complet."""
    start_time = time.time()
    
    cmd = [
        "python", f"{CODEBASE}/EXO_06_CARRIER.py",
        "--drive-root", DRIVE_ROOT,
        "--project-name", PRODUCTION_CONFIG["project_name"],
        "-v"
    ]
    
    # Options
    if PRODUCTION_CONFIG.get('components_dir'):
        cmd.extend(["--components-dir", PRODUCTION_CONFIG['components_dir']])
    
    if PRODUCTION_CONFIG.get('output_dir'):
        cmd.extend(["--output-dir", PRODUCTION_CONFIG['output_dir']])
    
    if PRODUCTION_CONFIG.get('production_plan'):
        cmd.extend(["--production-plan", PRODUCTION_CONFIG['production_plan']])
    
    if not PRODUCTION_CONFIG.get('use_rife'):
        cmd.append("--no-rife")
    
    if not PRODUCTION_CONFIG.get('use_upscale'):
        cmd.append("--no-upscale")
    
    if not PRODUCTION_CONFIG.get('use_gpu'):
        cmd.append("--cpu-only")
    
    print("╔" + "═" * 60 + "╗")
    print("║          AIRCRAFT CARRIER — PRODUCTION START              ║")
    print("╚" + "═" * 60 + "╝")
    print(f"\nProject: {PRODUCTION_CONFIG['project_name']}")
    print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("\n" + "-" * 60)
    
    # Exécution avec output en temps réel
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    
    for line in process.stdout:
        print(line, end='')
    
    process.wait()
    
    elapsed = time.time() - start_time
    
    print("\n" + "-" * 60)
    
    if process.returncode == 0:
        print(f"\n✅ PRODUCTION COMPLÈTE en {elapsed/60:.1f} minutes")
        return True
    else:
        print(f"\n❌ PRODUCTION ÉCHOUÉE (code: {process.returncode})")
        return False

# Lancer seulement si validation OK
if validation_ok:
    success = run_carrier()
else:
    print("⚠ Corrigez les erreurs avant de lancer la production")

## 📁 Vérification Outputs

In [ ]:
from pathlib import Path

output_dir = Path(PRODUCTION_CONFIG.get('output_dir') or f"{UNIT_ROOT}/OUT_FINAL")
project_name = PRODUCTION_CONFIG['project_name']

print("=== Fichiers Générés ===")
print(f"Dossier: {output_dir}\n")

expected_files = [
    (f"FINAL_OUTPUT_{project_name}.mp4", "H.265 MP4"),
    (f"FINAL_OUTPUT_{project_name}.mov", "ProRes MOV"),
    (f"thumbnail_{project_name}.png", "Thumbnail"),
    ("carrier_report.json", "Rapport"),
]

for filename, desc in expected_files:
    filepath = output_dir / filename
    if filepath.exists():
        size_mb = filepath.stat().st_size / (1024 * 1024)
        print(f"✓ {desc}: {filename} ({size_mb:.1f} MB)")
    else:
        print(f"✗ {desc}: {filename} (non trouvé)")

In [ ]:
# Inspecter le rapport
import json

report_path = output_dir / "carrier_report.json"

if report_path.exists():
    with open(report_path) as f:
        report = json.load(f)
    
    print("=== Carrier Report ===")
    print(f"\nStatus: {report.get('status')}")
    print(f"Project: {report.get('project')}")
    print(f"Timestamp: {report.get('timestamp')}")
    
    print(f"\nPipeline Stages:")
    for stage, info in report.get('pipeline', {}).items():
        status = info.get('status', 'N/A')
        print(f"  {stage}: {status}")
    
    print(f"\nOutputs:")
    for ftype, fpath in report.get('outputs', {}).items():
        print(f"  {ftype}: {fpath}")
    
    if report.get('warnings'):
        print(f"\nWarnings: {len(report['warnings'])}")
        for w in report['warnings'][:5]:
            print(f"  - {w}")
else:
    print("Rapport non trouvé")

## 🎬 Preview Output

In [ ]:
# Afficher la thumbnail
from IPython.display import Image, display

thumbnail_path = output_dir / f"thumbnail_{project_name}.png"

if thumbnail_path.exists():
    print("=== Thumbnail Preview ===")
    display(Image(filename=str(thumbnail_path), width=800))
else:
    print("Thumbnail non trouvée")

In [ ]:
# Infos vidéo finale
from final_encoder import FinalEncoder

encoder = FinalEncoder(verbose=True)

mp4_path = output_dir / f"FINAL_OUTPUT_{project_name}.mp4"

if mp4_path.exists():
    info = encoder.get_video_info(mp4_path)
    if info:
        print("=== Vidéo Finale ===")
        print(f"  Résolution: {info['width']}x{info['height']}")
        print(f"  FPS: {info['fps']}")
        print(f"  Durée: {info['duration']:.2f}s ({info['duration']/60:.1f} min)")
        print(f"  Codec: {info['codec']}")
        print(f"  Taille: {info['size_bytes'] / (1024*1024*1024):.2f} GB")
        print(f"  Bitrate: {info['bitrate'] / 1_000_000:.1f} Mbps")
else:
    print("Vidéo finale non trouvée")

## 📊 Rapport de Production

In [ ]:
# Générer rapport résumé
from datetime import datetime

print("╔" + "═" * 60 + "╗")
print("║              PRODUCTION SUMMARY REPORT                      ║")
print("╚" + "═" * 60 + "╝")

print(f"\nProject: {project_name}")
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

if mp4_path.exists() and info:
    print(f"\n📹 Output:")
    print(f"   {info['width']}x{info['height']} @ {info['fps']} FPS")
    print(f"   Duration: {info['duration']/60:.1f} minutes")
    print(f"   Size: {info['size_bytes'] / (1024*1024*1024):.2f} GB")
    print(f"   Bitrate: {info['bitrate'] / 1_000_000:.1f} Mbps")

print(f"\n📁 Files:")
for filename, desc in expected_files:
    filepath = output_dir / filename
    if filepath.exists():
        print(f"   ✓ {filepath}")

print(f"\n🚀 Ready for distribution!")

---
## ✅ Production Complete

Les fichiers finaux sont prêts:
- **MP4 H.265**: Pour distribution YouTube/réseaux
- **MOV ProRes**: Pour archivage haute qualité
- **Thumbnail**: Pour miniature de publication

**Prochaines étapes**:
1. Vérifier la qualité visuelle
2. Uploader sur YouTube/Vimeo
3. Archiver le ProRes sur stockage long terme